# Importing

In [1]:
from utilities.Functionnalities import *
import warnings
warnings.filterwarnings("ignore")

# Entries

In [2]:
dateObservation = 24
nbPool = 6
star_time = "2021-07-21"
hour = "00:00:00"
ending_time = "2021-07-24"

# Download Node Modules for nodeJs
### npm install ccxt
### npm install csv-writer

In [ ]:
install_package_if_needed("ccxt")
install_package_if_needed("csv-writer")

# 1] Data Collection

### Les Variables

In [3]:
data = {'current_date': pd.to_datetime(['2020-01-02']),
        'pool': [['BNB', 'ATD', 'ACC']]}
dff = pd.DataFrame(data)
dff.set_index('current_date', inplace=True)
dff.drop(dff.index, inplace=True)

command = 'node ./database/dl_for_quick_analysis.js'
path = "./database/quick_analysis"
ennDate = f"{ending_time} {hour}"
sttDate = f"{star_time} {hour}"
start_date = datetime.strptime(sttDate, "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime(ennDate, "%Y-%m-%d %H:%M:%S")

current_date = start_date

# Define the maximum number of threads
MAX_THREADS = 10

# Semaphore to control the concurrency
semaphore = threading.Semaphore(MAX_THREADS)

### Download all the OHLCV

In [ ]:
def downloadingDate(current_date):
    # Acquire the semaphore
    semaphore.acquire()

    execute_terminal_command2(command, current_date.strftime("%Y-%m-%d"), hour)
    remove_non_csv_files(path)

    # Release the semaphore
    semaphore.release()

In [ ]:
try:
    shutil.rmtree(path)
except:
    traceback.format_exc()
threads = []
while current_date <= end_date:
    thread = threading.Thread(target=downloadingDate, args=(current_date,))
    thread.start()
    threads.append(thread)
    current_date += timedelta(days=1)

# Wait for all threads to finish
for thread in threads:
    thread.join()
print("Process is done!")

### Extraire all the pair names

In [ ]:
pair_names = []

# Get the list of file names in the directory
file_names = os.listdir(path)

# Process each file name
for file_name in file_names:
    # Extract the pair name before the '$' character
    pair_name = file_name.split('$')[0]

    # Add the pair name to the list if it's not already present
    if pair_name not in pair_names:
        pair_names.append(pair_name)

# Print the list of unique pair names
print(pair_names)

### Create the csv files for all the dates

In [ ]:
pair_data = {}
# Process each file name
for file_name in file_names:
    # Extract the pair name before the '$' character
    pair_name = file_name.split('$')[0]

    # Check if the pair name is already in the dictionary
    if pair_name in pair_data:
        pair_data[pair_name].append(file_name)
    else:
        pair_data[pair_name] = [file_name]

# Iterate over the pair names and their corresponding files
for pair_name, files in pair_data.items():
    # Create a new file for the pair name
    output_file = os.path.join(path, f"{pair_name}.csv")

    # Open the output file in write mode
    with open(output_file, 'w', newline='') as csv_out:
        writer = csv.writer(csv_out)

        # Write the header to the output file
        writer.writerow(['date', 'open', 'high', 'low', 'close', 'volume'])

        # Iterate over the files for the pair name
        for file in files:
            # Open each input file
            with open(os.path.join(path, file), 'r') as csv_in:
                reader = csv.reader(csv_in)
                next(reader)  # Skip the header row

                # Write the data rows to the output file
                writer.writerows(reader)

    print(f"Created file: {output_file} with merged data and header.")

### Remove rest of csv files

In [ ]:
# Iterate over the file names
for file_name in file_names:
    # Check if the file name contains the '$' character
    if '$' in file_name:
        # Create the file path
        file_path = os.path.join(path, file_name)

        # Remove the file
        os.remove(file_path)
        print(f"Deleted file: {file_path}")

### Remove redudancy in the csv files

In [ ]:
# Get the list of file names in the directory
file_names = os.listdir(path)

# Iterate over the file names
for file_name in file_names:
    # Create the file path
    file_path = os.path.join(path, file_name)

    # Read the contents of the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Remove repeated rows and keep only the first occurrence
    unique_lines = []
    unique_set = set()
    for line in lines:
        if line not in unique_set:
            unique_lines.append(line)
            unique_set.add(line)

    # Write the unique lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(unique_lines)

    print(f"Removed repeated rows from file: {file_path}")

# 2] Data Preprocessing

In [ ]:
file_list = [f for f in listdir(path) if isfile(join(path, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(path + "/" + file)
print("All data loaded")

In [ ]:
# Convert the start and end times to datetime objects
start = datetime.strptime(star_time, "%Y-%m-%d")
end = datetime.strptime(ending_time, "%Y-%m-%d")
# Define the timedelta for incrementing the date
delta = timedelta(days=1)
# Loop through the dates between start and end
while start <= end:
    current_date = start.strftime("%Y-%m-%d")
    # Filter the dataframes to keep only the specific date
    filtered_dataframes = {}
    for key, df in df_list.items():
        filtered_df = df.loc[df.index.date == pd.to_datetime(start).date()]
        filtered_dataframes[key] = filtered_df
    df_metric = get_analyisis_from_window(df_list,dateObservation).sort_values(by="volume_evolution", ascending=False)
    dfVe = df_metric.iloc[:nbPool]
    market = list(dfVe.index)
    dff.loc[datetime.strptime(current_date, "%Y-%m-%d")] = [list(dfVe['volume_evolution'].index)]
    start += delta

### Save the Pools By day dataframe

In [ ]:
dff.to_csv('./database/pools.csv',header=False)

# 3] Generate All Combinations

In [4]:
crypto = {}
deltaHours = ["2h","4h","8h","12h"]
Ni = ["N","N-1","N-2"]
array1 = []
with open('database/pools.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        timestamp = row[0]
        elements = row[1].strip('[]').split(', ')
        array1.append((timestamp, elements))
combinations = list(itertools.product(array1, deltaHours, Ni))
combinations = [list(item) for item in combinations]

# 4] Cocotier Process

In [5]:
def get_historical_klines(x,deltahour,sttDate,ennDate):
    # Open the CSV file
    file_path = f"./database/quick_analysis/{x}-USDT.csv"
    df = pd.read_csv(file_path)

    # Convert sttDate and ennDate to datetime objects
    stt_date = pd.to_datetime(sttDate, format="%Y-%m-%d %H:%M:%S")
    enn_date = pd.to_datetime(ennDate, format="%Y-%m-%d %H:%M:%S")

    # Convert the 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'], unit='ms')

    # Filter the dataframe based on the start and end dates
    mask = (df['date'] >= stt_date) & (df['date'] <= enn_date)
    filtered_df = df.loc[mask]

    # Extract the delta hour value from the string (e.g., '2h', '4h')
    delta_hours = int(re.findall(r'\d+', deltahour)[0])

    # Create a time range at delta hour intervals
    time_range = pd.date_range(stt_date, enn_date, freq=f'{delta_hours}H')

    # Filter the dataframe to keep only the rows at delta hour intervals
    filtered_df = filtered_df[filtered_df['date'].isin(time_range)]

    # Create a new dataframe with the desired columns
    new_df = filtered_df[['date', 'open', 'close']].copy()
    new_df.columns = ['timestamp', f'{x.lower()}_open', f'{x.lower()}_close']

    return new_df

In [6]:
def cocotier(combination):
    semaphore.acquire()
    crypto = {}
    x = ""
    for elm in combination[0][1] :
        try :
            x = elm.replace("'","")

            crypto[x] = get_historical_klines(x,combination[1],sttDate,ennDate)
            crypto[x] = crypto[x].astype({x.lower()+ '_open': 'float64',x.lower()+ '_close': 'float64'})
            crypto[x] = crypto[x].set_index('timestamp')
        except Exception as ll:
            print(f"{ll}\n{x}!")
            traceback.format_exc()
    try :
        array_mauvais_shape = detection_mauvais_shape(crypto)
        # crypto = correction_shape(crypto, array_mauvais_shape)
        # for elm in array_mauvais_shape:
        #     crypto[elm]['timestamp'] = generation_date(crypto[elm], int(delta_hour[:1]))
        #     crypto[elm] = crypto[elm].set_index('timestamp')
        for i in array_mauvais_shape:
            del crypto[i]
        crypto = variationN(crypto, combination[2])
        crypto = coeffMulti(crypto)
        crypto = mergeCryptoTogether(crypto)
        crypto, maxis = botMax(crypto)
        crypto = botMaxVariation2(crypto, maxis)
        crypto = coeffMultiBotMax(crypto)
        coefMulti = coefmultiFinal(crypto)
        combination.append(coefMulti.tail(1).iloc[-1,-1])
    except Exception as ll:
        print(f"{ll}\n")
    semaphore.release()

In [7]:
# Launch a thread for each iteration
threads = []
combinations = list(combinations)
for combination in combinations:
    thread = threading.Thread(target=cocotier, args=(combination,))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

# 5] Presenting the First DataFrame

In [9]:
# Flatten the nested lists
flat_data = []
for combination in combinations:
    datetime = combination[0][0]
    pool = combination[0][1]
    deltahour = combination[1]
    Ni = combination[2]
    BotMax = combination[3]
    flat_data.append([datetime, deltahour, Ni, BotMax, pool])
df = pd.DataFrame(flat_data, columns=['datetime', 'deltahour', 'Ni', 'BotMax','pool'])

# 6] Produit Cumulée and final Dataframe

In [13]:
# Convert 'datetime' column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Group DataFrame by 'deltahour' and 'Ni', calculate product of 'BotMax'
df_grouped = df.groupby(['deltahour', 'Ni']).agg({'datetime': ['min', 'max'], 'BotMax': 'prod'}).reset_index()

# Rename columns
df_grouped.columns = ['deltahour', 'Ni', 'startDate', 'endingDate', 'BotMax']

# Convert 'startDate' and 'endingDate' columns to desired format
df_grouped['startDate'] = df_grouped['startDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_grouped['endingDate'] = df_grouped['endingDate'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [14]:
print(df_grouped)

   deltahour   Ni            startDate           endingDate    BotMax
0        12h    N  2021-07-21 00:00:00  2021-07-24 00:00:00  0.918403
1        12h  N-1  2021-07-21 00:00:00  2021-07-24 00:00:00  0.918403
2        12h  N-2  2021-07-21 00:00:00  2021-07-24 00:00:00  0.918403
3         2h    N  2021-07-21 00:00:00  2021-07-24 00:00:00  1.103001
4         2h  N-1  2021-07-21 00:00:00  2021-07-24 00:00:00  1.103001
5         2h  N-2  2021-07-21 00:00:00  2021-07-24 00:00:00  1.103001
6         4h    N  2021-07-21 00:00:00  2021-07-24 00:00:00  1.120594
7         4h  N-1  2021-07-21 00:00:00  2021-07-24 00:00:00  1.120594
8         4h  N-2  2021-07-21 00:00:00  2021-07-24 00:00:00  1.120594
9         8h    N  2021-07-21 00:00:00  2021-07-24 00:00:00  0.978497
10        8h  N-1  2021-07-21 00:00:00  2021-07-24 00:00:00  0.978497
11        8h  N-2  2021-07-21 00:00:00  2021-07-24 00:00:00  0.978497
